In [1]:
import numpy as np
import math
import cv2
import tdmclient.notebook
import utils

In [3]:
await tdmclient.notebook.start()

In [4]:
# await tdmclient.notebook.stop()

In [5]:
# %%run_aseba
# onevent run
#     motor.left.target = 300
#     motor.right.target = 300

# onevent turnleft
#     motor.left.target = -300
#     motor.right.target = 300

# onevent turnright
#     motor.left.target = 300
#     motor.right.target = -300

# onevent stop
#     motor.left.target = 0
#     motor.right.target = 0

Define possible objects and marker colors

In [6]:
ball_blue = [[103, 196, 130], [110, 255, 224]]
red = [[0, 106, 180], [9, 196, 229]]
brown = [[10, 61, 123], [16, 198, 167]]
green = [[31, 78, 114], [65, 199, 193]]
yellow = [[23, 170, 154], [31, 255, 248]]
blue = [[75, 116, 167], [100, 255, 255]]

In [7]:

robot1 = {"front":1, "back":2, "f_hsv": red, "b_hsv": green, "front_loc":(), "back_loc":()}
# robot2 = {"front":3, "back":4, "f_hsv": [[], []], "b_hsv": [[], []], "front_loc":(), "back_loc":()}
ball = {"center":5, "hsv": ball_blue, "loc":()}
goal1 = {"center": 7, "hsv": yellow, "loc":()}
goal2 = {"center": 6, "hsv": blue, "loc":()}
stadium = {"top_left_hsv": brown, "loc": []}

In [8]:
field = {
    "ball": ball,
    "goal1": goal1,
    "goal2": goal2,
}

Define what robots are in the game

In [9]:
robots = {
    "robot1": robot1,
    # "robot2": robot2
}


Define events in aseba

In [10]:
%%run_aseba

onevent walk300
  motor.left.target = 300
  motor.right.target = 300
  
onevent walk200
  motor.left.target = 200
  motor.right.target = 200

onevent walk100
  motor.left.target = 100
  motor.right.target = 100

onevent walk50
  motor.left.target = 50
  motor.right.target = 50

onevent left_turn
  motor.left.target = 100
  motor.right.target = -100

onevent right_turn
  motor.left.target = -100
  motor.right.target = 100

onevent stop
  motor.left.target = 0
  motor.right.target = 0

In [11]:
def move_to_shooting_point(angle_of_rotation, distance_to_point, rotation_threshold, distance_threshold):
    # print(angle_of_rotation)
    if ( abs(angle_of_rotation) > rotation_threshold) and (distance_to_point > distance_threshold):
        # print("inside 1", " ", angle_of_rotation)
        if(angle_of_rotation > 0):
            print("direita")
            send_event("right_turn")
        else:
            print("esquerda")
            send_event("left_turn")
    else:
        # print("inside 2")
        if(distance_to_point > distance_threshold) :
            "andar"
            send_event("walk300")
        else:
            send_event("stop")

Variables

In [12]:
COLOR_TOLERANCE: int = 20
DIST_BALL: int = 15
RT_TRESH: int = 20
DT_TRESH: int = 20


Unchecked AnnAssign(target=Name(id='COLOR_TOLERANCE', ctx=Store()), annotation=Name(id='int', ctx=Load()), value=Constant(value=20), simple=1)
Unchecked AnnAssign(target=Name(id='DIST_BALL', ctx=Store()), annotation=Name(id='int', ctx=Load()), value=Constant(value=15), simple=1)
Unchecked AnnAssign(target=Name(id='RT_TRESH', ctx=Store()), annotation=Name(id='int', ctx=Load()), value=Constant(value=20), simple=1)
Unchecked AnnAssign(target=Name(id='DT_TRESH', ctx=Store()), annotation=Name(id='int', ctx=Load()), value=Constant(value=20), simple=1)


In [13]:
cap = cv2.VideoCapture(0)

In [14]:
prev_shoot_x = None
prev_shoot_y = None
shooting: bool = False

Unchecked AnnAssign(target=Name(id='shooting', ctx=Store()), annotation=Name(id='bool', ctx=Load()), value=Constant(value=False), simple=1)


Control random noise

In [ ]:
buffer_size = 10  # Number of recent points to consider
shooting_point_buffer = []

In [ ]:

def update_buffer(_new_x, _new_y, buffer, buffer_size):
    if isinstance(_new_x, (int, float)) and isinstance(_new_y, (int, float)):

        # Append the values to the buffer
        buffer.append((_new_x, _new_y))
        if len(buffer) > buffer_size:
            buffer.pop(0)
    else:
        print("Invalid shooting point: x or y is not a number")

    return buffer

In [ ]:
def calculate_median_buffer_point(buffer):
    if not buffer:
        return None, None

    # Separate x and y coordinates
    x_coords, y_coords = zip(*buffer)

    # Calculate median
    median_x = np.median(x_coords)
    median_y = np.median(y_coords)

    return int(median_x), int(median_y)

In [15]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    matrix = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

    cv2.imshow('real_frame', frame)
    # Process each robot's front and back points
    for robot, details in robots.items():
        # Detect front points
        f_lower_bound = np.array(details['f_hsv'][0])
        f_upper_bound = np.array(details['f_hsv'][1])
        front_coordinates = utils.detect_colored_dots_in_rgb(image, f_lower_bound, f_upper_bound)
        robots[robot]["front_loc"] = utils.safe_get(front_coordinates, 0, [])

        # Mark front points on the matrix
        for coord in front_coordinates:
            matrix[coord[0], coord[1]] = details['front']
        
        # Detect back points
        b_lower_bound = np.array(details['b_hsv'][0])
        b_upper_bound = np.array(details['b_hsv'][1])
        back_coordinates = utils.detect_colored_dots_in_rgb(image, b_lower_bound, b_upper_bound)
        robots[robot]["back_loc"] = utils.safe_get(back_coordinates, 0, [])

        # Mark back points on the matrix
        for coord in back_coordinates:
            matrix[coord[0], coord[1]] = details['back']
        


    # Find the coordinates of goals and ball
    for feature, details in field.items():
        lower_bound = np.array(details['hsv'][0])
        upper_bound = np.array(details['hsv'][1])
        coordinates = utils.detect_colored_dots_in_rgb(image, lower_bound, upper_bound)
        field[feature]["loc"] = utils.safe_get(coordinates, 0, [])

        for coord in coordinates:
            matrix[coord[0], coord[1]] = details['center']
            # print(feature, coord[0], coord[1], details['center'])
    

    #___________________________________________________________TEST_PART______________________________________

    color_map = {
        # 0: (0, 0, 0),       # Black (assuming 0 is the background or no dot)
        1: (255, 0, 0),     # Red
        2: (0, 255, 0),     # Green
        3: (0, 0, 255),     # Blue
        4: (255, 255, 0),   # Cyan
        5: (255, 0, 255),   # Magenta
        6: (0, 255, 255),   # Yellow
        7: (255, 255, 255)  # White
    }

    # Create an empty frame with 3 color channels
    colored_frame = np.zeros((matrix.shape[0], matrix.shape[1], 3), dtype=np.uint8)

    # Draw colored circles in the frame based on matrix values
    for y in range(matrix.shape[0]):
        for x in range(matrix.shape[1]):
            value = matrix[y, x]
            if value in color_map:
                cv2.circle(colored_frame, (x, y), 10, color_map[value], 14)  # Circle with radius 10
    

    try:
        shoot_x, shoot_y = utils.ball_shooting_point(field["ball"]["loc"], field["goal2"]["loc"], DIST_BALL, False)
        # if prev_shoot_x == None and shoot_x != None:
        #     prev_shoot_x = shoot_x
        #     prev_shoot_y = shoot_y
        # else:
        #     shoot_x = prev_shoot_x
        #     shoot_y = prev_shoot_y
            
        # print(shoot_x, shoot_y)
        cv2.line(colored_frame, (shoot_x, shoot_y)[::-1], field["goal2"]["loc"][::-1], (255, 255, 255))
        cv2.line(colored_frame, robots["robot1"]["back_loc"][::-1], (shoot_x, shoot_y)[::-1], (0, 255, 255))

        r1x, r1y, r1orientation = utils.calculate_robot_position_and_orientation(matrix, robots["robot1"]["front"], robots["robot1"]["back"])
        angle_of_rotation_sp = utils.get_angle_of_rotation((r1x, r1y, r1orientation), (shoot_x, shoot_y))
        # print(angle_of_rotation_sp)
        dist_shoot_sp = utils.get_distance_between_two_points((r1x, r1y), (shoot_x, shoot_y))
        
        print(angle_of_rotation_sp, dist_shoot_sp, shooting)

        if dist_shoot_sp > DT_TRESH and ~shooting:
            move_to_shooting_point(angle_of_rotation_sp, dist_shoot_sp, RT_TRESH, DT_TRESH)
        else:
            angle_of_rotation_ball = utils.get_angle_of_rotation((r1x, r1y, r1orientation), field["goal2"]["loc"])
            dist_shoot_ball = utils.get_distance_between_two_points((r1x, r1y), ball_fixed)
            move_to_shooting_point(angle_of_rotation_ball, dist_shoot_ball, RT_TRESH+10, DT_TRESH+10)

        if dist_shoot_sp < DT_TRESH and ~shooting:
            shooting = True
            ball_fixed = field["ball"]["loc"]
            print("switching")
            
    except:
        pass
    colored_frame = cv2.cvtColor(colored_frame, cv2.COLOR_RGB2BGR)
    # Display the frame
    cv2.imshow('Frame', colored_frame)

    # Wait for 'q' key to quit
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    #___________________________________________________________TEST_PART______________________________________
    

cv2.destroyAllWindows()

-116.85573886130948 49.73932046178355 False
esquerda
-113.60411504291426 80.49844718999243 False
esquerda
-113.60411504291426 80.49844718999243 False
esquerda
164.7154200693928 333.5655857548857 False
direita
-4.274468692884227 161.24515496597098 False
-14.537869870164002 170.07351351694948 False
-19.090593736564102 178.08986495586996 False
-24.339127281963954 178.46568297574746 False
esquerda
-30.94678790604266 168.86088949191284 False
esquerda
-26.43423912237327 164.92422502470643 False
esquerda
-23.11585159412948 162.5607578722491 False
esquerda
-17.84615916737812 166.1114083980989 False
-13.347585057844697 34.52535300326414 False
-96.50895644050252 47.43416490252569 False
esquerda
-173.92384223534918 26.40075756488817 False
esquerda
-20.629253865379752 21.02379604162864 False
esquerda
-111.58093619449926 16.1245154965971 False
92.65419551340665 3.1622776601683795 False
104.30027744918561 4.47213595499958 False
-100.69023827649912 8.54400374531753 False
-107.57664929289051 7.2111025

In [16]:
cap.release()

In [17]:
send_event("stop")

Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2032.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\infwo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tdmclient\tcp.py", line 73, in run
    packet = self.read_packet()
             ^^^^^^^^^^^^^^^^^^
  File "C:\Users\infwo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tdmclient\tcp.py", line 66, in read_packet
    raise error
  File "C:\Users\infwo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tdmclient\tcp.py", line 59, in read_packet
    packet_len = self.read_uint32()
                 ^^^^^^^^^^^^^^^^^^
  File "C:\Users\infwo\AppData\Local\Packages\Pytho